<a href="https://colab.research.google.com/github/geraldomsousaj/industrial-challenge-EDF/blob/importanceFactor/3_0_geraldomsousaj_hydraulic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openturns 
import pandas as pd
import numpy as np
import openturns as ot

In [2]:
# defining the constants
L = 5000 # length of section (m)
B = 300 # watercourse width (m)
Zb = 55.5 # shoreline elevation (m)

In [3]:
def function_flood(X):
    Q, Ks, Zv, Zm = X
    alpha = (Zm - Zv)/L
    H = (Q/(Ks * B * np.sqrt(alpha)))** (3.0/5.0) 
    S = H + Zv - Zb
    return [S]

input_dimension = 4
g = ot.PythonFunction(input_dimension , 1, function_flood)

In [4]:
# Create the marginal distributions of the parameters
dist_Q = ot.TruncatedDistribution(ot.Gumbel(558., 1013.), 0, ot.TruncatedDistribution.LOWER)
dist_Ks = ot.TruncatedDistribution(ot.Normal(30.0, 7.5), 0, ot.TruncatedDistribution.LOWER)
dist_Zv = ot.Triangular(49, 50 , 51)
dist_Zm = ot.Triangular(54, 55 , 56)

In [5]:
X = ot.ComposedDistribution([dist_Q, dist_Ks, dist_Zv, dist_Zm])
inputRV = ot.RandomVector(X)

In [6]:
# Run simulations
N = 100000
S = []
for i in range(N):
    S.append(function_flood(inputRV.getRealization()))

In [7]:
S_df = pd.DataFrame(S, columns=['S'])
print('% of positive values: ' + str(S_df[S_df['S'] > 0].shape[0] / N *100)+"%")

% of positive values: 1.18%


In [8]:
df2 = S_df[S_df['S'] > 0]  

In [9]:
df2.sort_values(by='S', ascending=False).head(20)

,S
44871,26.261403
31510,19.380357
69886,12.032397
53839,9.960293
37320,9.552282
11143,8.449689
53743,7.441275
85957,7.385330
97123,6.827439
95301,5.584294


In [33]:
q = [0.95, 0.96, 0.965, 0.97, 0.975, 0.98, 0.985, 0.99, 0.995, 0.996, 0.997, 0.998]
for i in q:
    print('Quantile {}%: {:0.4f}'.format(i*100, df2.quantile(i).S))


Quantile 95.0%: 2.5694
Quantile 96.0%: 2.8290
Quantile 96.5%: 2.9600
Quantile 97.0%: 3.2717
Quantile 97.5%: 3.5275
Quantile 98.0%: 3.9323
Quantile 98.5%: 4.1829
Quantile 99.0%: 5.0585
Quantile 99.5%: 7.5472
Quantile 99.6%: 8.7628
Quantile 99.7%: 9.7412
Quantile 99.8%: 11.2906
